# 설명

### 크롤링된 태그들을 다양한 형태의 테이블로 만들어주는 프로그램

#### 사용방법
#### 1. C:\Users\acorn\weather_bigdata\seperated_tag\ 에  해시태그명(ex:경복궁)으로 폴더를 만들고 
####     해당태그로 크롤링된 분할 태그파일들을 옮긴다

#### 2. 프로그램을 실행시키고  키워드 입력에 해시태그명을 적으면 자동으로 다양한 형태의 테이블 파일(.csv)이 만들어짐

In [2]:
import os
import csv
import codecs
import sys
import re
import pandas as pd
import numpy as np

In [9]:
class TextMiner:
    def replace_all(self,string, substitutions):
        substrings = sorted(substitutions, key=len, reverse=True)
        regex = re.compile('|'.join(map(re.escape, substrings)))
        return regex.sub(lambda match: substitutions[match.group(0)], string)

    def modifyHash(self,hash_str):
        substitutions = {"[":"", "]":"", "#":"", ",":"", "'":""}
        modifed_str = self.replace_all(hash_str, substitutions)
        str_list = modifed_str.split(" ")
        return self.setStrDict(str_list)

    # 하나의 해쉬 리스트에 들어있는 낱말과 카운트
    def setStrDict(self,str_list):
        dic = {}

        for item in str_list:
            if item=="":
                continue
            else:
                if item in dic:
                    dic[item]+=1
                else:
                    dic[item]=1
        return dic

    def writeDicCsv(self, dic, date, writer):
        for row in dic.items(): # dict의 (key, value) 쌍을 복사한 list를 return
            overall = row + (date,)  # 단어, 빈도수, 날짜 합친 튜플
            writer.writerow(overall)
            
            
    def miningBegin(self,keyword, merged_file_name_path, folder_path):
        f = open(merged_file_name_path, 'r', encoding='utf-8')
        info_list = csv.reader(f)

        new_file_path = folder_path # (word,freq,date).csv
        new_file_name = keyword+'_(word,freq,date).csv'

        new_file_path_name = new_file_path + new_file_name

        file = open(new_file_path_name, 'w', encoding='utf-8')
        file.write('word,freq,date\n')
        writer = csv.writer(file)

        print("해시태그 추출 및 불필요한 단어 제거중...")
        for row in info_list:
            dic = self.modifyHash(row[2])
            self.writeDicCsv(dic,row[1], writer)

        f.close()
        file.close()

In [14]:
class Processor:
    def __init__(self, keyword, seperated_file_path, merged_file_path, complete_file_path):
        self.keyword = keyword
        self.seperated_file_path = seperated_file_path
        self.merged_file_path = merged_file_path
        self.merged_file_name = keyword+'_merged.csv'
        self.merged_file_path_name = self.merged_file_path + self.merged_file_name # merge 할 파일 경로 + 이름
        self.folder_path = complete_file_path+self.keyword+"/"
        self.remove_list = [keyword,'소통','소통해요','소통환영','선팔','맞팔','셀스타그램','셀카','셀피','선팔하면맞팔',
                            'f4f', 'l4l', '일상', 'daily', '좋아요','좋아요반사','인친','fff',
                            '데일리','selfie','selca','서울','seoul', '팔로우', 'ootd', '오오티디',
                           '인스타그램','instagram','데일리그램','follow','like-4-like', 
                            'follow-4-follow','follow','맞팔해요','맞팔환영','인친해요','인친환영',
                           'korea','한국', '얼스타그램','아이폰','iphone','데일리룩','dailylook','instadaily',
                           '선팔맞팔','lfl', 'instagood', '셀', '셀기꾼', '인스타', '인스타데일리', '일상스타그램', 'followme',
                           '댓글', '어제','오늘', '내일', 'selstagram','팔로워','패션','like4like','첫줄','술스타그램',
                            'follow4follow', '친스타그램', '패션스타그램']
        
        # 사전을 파일로 만들어서 필터링 ?
        
        if not os.path.exists(self.folder_path): 
            os.makedirs(self.folder_path)

    # 여러대의 크롤러에 의해 수집된 TAG 파일(id , date, hashtag, overall)들을 하나로 합친다
    def tag_file_merging(self): 
        file_path = self.seperated_file_path + self.keyword + "//"
        file_list = os.listdir(file_path) # directory 내 파일 목록
        merged_file = codecs.open(self.merged_file_path_name, 'w', 'utf-8') # merge할 파일 생성
        
        writer = csv.writer(merged_file)

        for file_name in file_list : # merging
            current_file = file_path + file_name # 현재 파일 경로 + 이름

            with codecs.open(current_file, 'r', 'utf-8') as f:
                reader = csv.reader(f) # 현재 파일 읽기

                for line in reader: #  현재 파일의 내용을 merge할 파일로 옮김
                    writer.writerow(line)
                    
        print("분리된 %s 데이터 파일 병합 완료!" %self.keyword)
        merged_file.close()    
    
    # overall 열(id + date + hashtag가 합쳐진 문자열을 value로 갖는다)을 이용해 
    # 중복된 항목들을 제거한다 overall 열을 삭제한다 
    def duplicate_remove(self):
        sys._enablelegacywindowsfsencoding() #  파일시스템 인코딩을 예전 방식으로 바꾸는 것
        df = pd.read_csv(self.merged_file_path_name, header=None)
        df_modified = df.drop_duplicates(subset=[3]) # overall 열에서 겹치는 부분 지우기
        del df_modified[3] # overall열 지우기
        target =['[]']
        df_modified = df_modified[~df_modified[2].isin(target)] # 해시태그 빈칸인거 지우기
        df_modified.columns = ['user_id', 'date', 'hashtag']
        df_modified.to_csv(self.merged_file_path_name, index=False) # mode default = 'w'
        print("병합된 %s 데이터 파일 중복제거 완료!" %self.keyword)
        
    
    def make_word_freq_date(self): #word,freq,date
        tm = TextMiner()
        tm.miningBegin(self.keyword, self.merged_file_path_name, self.folder_path)
        print("word,freq,date 파일 완료!")
        print("=>저장 위치 : %s " %self.folder_path)
        
    
    def make_word_freq(self): #word,freq
        file_name = self.keyword +'_(word,freq,date).csv'
        
        df = pd.read_csv(self.folder_path + file_name)
        df = df.groupby(['word'],as_index=False).sum()
        df = df[~df['word'].isin(self.remove_list)]
        new_df = df[df.freq>100]

        file_path = self.folder_path
        file_name = keyword+'_(word,freq).csv'
        file_path_name = file_path+file_name
        
        new_df.to_csv(file_path+file_name, index=None)
        print("word,freq 파일 완료!")
        print("=>저장 위치 : %s" %self.folder_path)
        
    def make_top150(self):
        file_name = self.keyword +'_(word,freq,date).csv'

        df = pd.read_csv(self.folder_path + file_name)
        df = df.groupby(['word'],as_index=False).sum()
        df = df[~df['word'].isin(self.remove_list)]
        new_df = df[df.freq>100]
        final_df = new_df.nlargest(150, 'freq')
        
        file_path = self.folder_path
        file_name = keyword+'_top150.csv'
        file_path_name = file_path+file_name
        
        final_df.to_csv(file_path_name, index=None)
        print("top150 파일 완료!")
        print("=>저장 위치 : %s" %self.folder_path)
    
    
    #날짜별 나온 단어와 횟수
    def make_word_by_date(self):
        file_name = self.keyword +'_(word,freq,date).csv'
        
        df = pd.read_csv(self.folder_path + file_name)
        new_df = df.groupby(['date','word'], as_index=False).sum()
        #new_df = new_df[new_df.freq>0]
        final_df = new_df[~new_df['word'].isin(self.remove_list)]
       

        file_path = self.folder_path
        file_name = keyword+'_(word_by_date).csv'
        file_path_name = file_path+file_name
        
        final_df.to_csv(file_path_name, index=None)
        print("word_by_date 완료!")
        print("=>저장 위치 : %s"  %self.folder_path)
        
    #top150에 있는 단어가 아니면 필터링
    def make_word_by_date_filtering(self):
        file_name1 = keyword +'_top150.csv'
        file_name2 = keyword+'_(word_by_date).csv'
        saved_file_name = keyword+'_word_by_date_filtering.csv'

        # 빈도수 10 이상인 단어만
        df = pd.read_csv(self.folder_path + file_name1)
 
        df2 = pd.read_csv(self.folder_path + file_name2)
        new_df = df2.groupby(['date','word'], as_index=False).sum()
        final_df = new_df[new_df['word'].isin(df['word'])]
        final_df.to_csv(self.folder_path + saved_file_name, index=None) 
        print("word_by_date_filtering 완료!")
        print("=>저장 위치 : %s"  %self.folder_path)
    
    def make_date_gatsu(self): #date, posting_gatsu
        df = pd.read_csv(self.merged_file_path_name)
        del df['user_id']
        del df['hashtag']
        df['posting_gatsu'] = np.array([1] * len(df))
        df = df.groupby('date').sum()
        
        file_path = self.folder_path
        file_name = keyword+'_(date,posting_gatsu).csv'
        file_path_name = file_path+file_name
        
        df.to_csv(file_path_name)
        print("date,gatsu 파일 완료!")
        print("=>저장 위치 : %s"  %self.folder_path)
        
    def main(self):
        self.tag_file_merging()
        self.duplicate_remove()
        self.make_word_freq_date()
        #self.make_word_freq()
        self.make_top150()
        self.make_word_by_date()
        self.make_word_by_date_filtering()
        self.make_date_gatsu()
        print("--------------전체 작업 종료--------------")

In [17]:
keyword = input("키워드 입력 : ")
seperated_file_path = 'C:/Users/acorn/weather_bigdata/seperated_tag/'
merged_file_path = 'C:/Users/acorn/weather_bigdata/merged_tag/'
complete_file_path = 'C:/Users/acorn/Desktop/result/'
#complete_file_path = 'C:/Users/acorn/weather_bigdata/complete/'

processor = Processor(keyword, seperated_file_path, merged_file_path, complete_file_path)
processor.main()

키워드 입력 : 제주도
분리된 제주도 데이터 파일 병합 완료!
병합된 제주도 데이터 파일 중복제거 완료!
해시태그 추출 및 불필요한 단어 제거중...
word,freq,date 파일 완료!
=>저장 위치 : C:/Users/acorn/Desktop/result/제주도/ 
top150 파일 완료!
=>저장 위치 : C:/Users/acorn/Desktop/result/제주도/
word_by_date 완료!
=>저장 위치 : C:/Users/acorn/Desktop/result/제주도/
word_by_date_filtering 완료!
=>저장 위치 : C:/Users/acorn/Desktop/result/제주도/
date,gatsu 파일 완료!
=>저장 위치 : C:/Users/acorn/Desktop/result/제주도/
--------------전체 작업 종료--------------
